In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pickle
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
test_df = pd.read_pickle('data/clean/combined/df_combined.p')

In [10]:
tweet_path = 'data/clean/twitter/full_tweet_df_v2.p'
meta_path = 'data/clean/meta/political_ads_w_metadata_v2.p'
parl_path = 'data/clean/parliament/transcripts_imputed_v2.p'

In [11]:
with open(tweet_path, 'rb') as a, open(meta_path, 'rb') as b, open(parl_path, 'rb') as c:
    tweet_df = pickle.load(a)
    meta_df = pickle.load(b)
    parl_df = pickle.load(c)

In [19]:
mps = pd.read_excel("/Users/mathiasbruun/Desktop/Uni/Master's Thesis/DCPA/mp_tally.xlsx", header = None)

In [15]:
mps = pd.read_csv('data/raw/parliament/MP_names_15_19.csv')

In [341]:
colmap_tw = {
        'full_text': 'doc',
        'name': 'full_name',
        'id_str': 'doc_id',
        'created_at': 'start_time',
        'description': 'tw_bio',
        'screen_name': 'tw_handle',
        'followers_count': 'tw_followers'
    }

colmap_meta = {
        'ad_creative_bodies': 'doc',
        'id': 'doc_id',
        'bylines': 'meta_sponsor',
        'ad_delivery_start_time': 'start_time',
        'ad_delivery_stop_time': 'end_time',
        'page_name': 'full_name',
        'name': 'wiki_name',
        'ad_snapshot_url': 'meta_url',
        'demographic_distribution': 'meta_demography',
        'delivery_by_region': 'meta_region',
        'estimated_audience_size': 'meta_audience',
        'publisher_platforms': 'meta_platforms',
        'spend': 'meta_spend',
        'currency': 'meta_currency',
        'impressions': 'meta_impressions',
        'ad_creative_link_titles': 'meta_titles',
        'ad_creative_link_captions': 'meta_captions',
        'ad_creative_link_descriptions': 'meta_descriptions',
        'birth_year': 'wiki_birth_year',
        'group_name': 'party',
        'electoral_region': 'wiki_electoral_region',
        'election_data_year': 'wiki_election_year',
        'education': 'wiki_edu',
        'personal_votes': 'wiki_votes'
    }

colmap_parl = {
        'text': 'doc',
        'group_name': 'party',
        'role': 'parl_role'
    }

In [342]:
tweet_df['source'] = 'twitter'
tweet_df = tweet_df.rename(
    columns = colmap_tw
)
tweet_df.head(2)

,tw_handle,full_name,tw_bio,tw_followers,doc_id,start_time,doc,source
0,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864,1570289445332459523,2022-09-15 05:52:32+00:00,@HrTousgaard @JTzsche @regionerne @Sygeplejera...,twitter
1,AndersKuhnau,Anders Kühnau,Regionsrådsformand i Region Midtjylland. Forma...,1864,1570069366816792576,2022-09-14 15:18:02+00:00,@JTzsche @regionerne @Sygeplejeraadet Du ved f...,twitter


In [343]:
meta_df['source'] = 'meta'
meta_df = meta_df.rename(
    columns = colmap_meta
)
meta_df.head(2)

,page_id,full_name,meta_sponsor,doc,start_time,meta_url,meta_demography,meta_region,meta_audience,meta_platforms,...,meta_descriptions,level,wiki_name,wiki_birth_year,party,wiki_electoral_region,wiki_election_year,wiki_edu,wiki_votes,source
0,132364993978468,Andreas Steenberg,Andreas Steenberg,[Hvor dum og uretfærdig kan vores udlændingelo...,2022-10-22,https://www.facebook.com/ads/archive/render_ad...,"[{'percentage': '0.026316', 'age': '55-64', 'g...","[{'percentage': '1', 'region': 'Central Denmar...","{'lower_bound': '100001', 'upper_bound': '5000...","[facebook, instagram]",...,NaN,politician,Andreas Steenberg,1983 (35 år),Radikale Venstre,Vestjylland (Kredsmandat),2019.0,cand.scient.pol.,4.444,meta
1,132364993978468,Andreas Steenberg,Andreas Steenberg,[Hvor dum og uretfærdig kan vores udlændingelo...,2022-10-22,https://www.facebook.com/ads/archive/render_ad...,"[{'percentage': '0.065789', 'age': '45-54', 'g...","[{'percentage': '0.230263', 'region': 'Central...","{'lower_bound': '50001', 'upper_bound': '100000'}","[facebook, instagram]",...,NaN,politician,Andreas Steenberg,1983 (35 år),Radikale Venstre,Vestjylland (Kredsmandat),2019.0,cand.scient.pol.,4.444,meta


In [344]:
parl_df['source'] = 'parliament'
parl_df = parl_df.rename(
    columns = colmap_parl
)
parl_df.head(2)

,first_name,last_name,full_name,party,parl_role,doc,year,day,time,start_time,end_time,duration,source
0,Lars Løkke,Rasmussen,Lars Løkke Rasmussen,"V,UFG,M",minister,Danmark er et godt land at leve i. Vi har bygg...,2009,10-06,12:04:01,2009-10-06T12:04:01,2009-10-06T12:50:39,0 days 00:46:38,parliament
1,Bjarne,Laustsen,Bjarne Laustsen,S,medlem,"Tak, hr. formand. Den sag, jeg vil bringe på b...",2009,10-07,13:04:25,2009-10-07T13:04:25,2009-10-07T13:06:31,0 days 00:02:06,parliament


In [345]:
df_concat = pd.concat([tweet_df, meta_df, parl_df])

In [346]:
df_concat.columns

Index(['tw_handle', 'full_name', 'tw_bio', 'tw_followers', 'doc_id',
       'start_time', 'doc', 'source', 'page_id', 'meta_sponsor', 'meta_url',
       'meta_demography', 'meta_region', 'meta_audience', 'meta_platforms',
       'meta_spend', 'meta_currency', 'meta_impressions', 'meta_titles',
       'end_time', 'meta_captions', 'meta_descriptions', 'level', 'wiki_name',
       'wiki_birth_year', 'party', 'wiki_electoral_region',
       'wiki_election_year', 'wiki_edu', 'wiki_votes', 'first_name',
       'last_name', 'parl_role', 'year', 'day', 'time', 'duration'],
      dtype='object')

In [347]:
#(
#    df_concat[['source', 'party', 'full_name']]
#        .sort_values(by = 'full_name')
#        .drop_duplicates(subset=['full_name'])
#).to_excel('politician_names.xlsx')

In [348]:
mapped_names = pd.read_excel('politician_names_coded.xlsx')

In [349]:
mapped_names = mapped_names.loc[mapped_names['mapping'].notna(), ['full_name', 'mapping']].set_index('full_name')

In [350]:
mapped_names

,mapping
full_name,
Aaja Chemnitz Larsen,Aaja Chemnitz
"Aaja Chemnitz Larsen, Inuit Ataqatigiit",Aaja Chemnitz
"Aaja Chemnitz, Inuit Ataqatigiit",Aaja Chemnitz
Aki Høegh-Dam,Aki-Matilda Høegh-Dam
Ane Halsboe-Larsen,Ane Halsboe-Jørgensen
...,...
karen lagoni,Karen Lagoni
malene busk,Malene Busk
melanie simick,Melanie Simick


In [351]:
df_concat['full_name'] = df_concat['full_name'].map(mapped_names['mapping']).fillna(df_concat['full_name'])

In [352]:
df = df_concat.copy()

In [353]:
df['party'].value_counts()

S                              62774
V                              53013
DF                             40235
EL                             34710
SF                             25689
KF                             21412
RV                             20403
LA                             18167
ALT                             8670
NB                              5164
Venstre                         4651
Socialistisk Folkeparti         2905
V,UFG                           2348
UFG                             1875
V,UFG,M                         1726
Det Konservative Folkeparti     1718
Socialdemokratiet               1701
SF,S                            1402
Nye Borgerlige                  1361
Radikale Venstre                1240
IA                              1223
Liberal Alliance                1036
KD                               813
Dansk Folkeparti                 738
FG                               643
Enhedslisten                     557
SIU                              507
J

In [374]:
party_map = {
    'Radikale Venstre': 'RV',
    'Venstre': 'V',
    'Socialistisk Folkeparti': 'SF',
    'Det Konservative Folkeparti': 'KF',
    'Socialdemokratiet': 'S',
    'Nye Borgerlige': 'NB',
    'Liberal Alliance': 'LA',
    'Dansk Folkeparti': 'DF',
    'Enhedslisten': 'EL',
    'Alternativet': 'ALT',
    'Inuit Ataqatigiit': 'IA',
    'Siumut': 'SIU',
    'Sambandsflokkurin': 'SP',
    'Javnaðarflokkurin': 'JF',
    'Tjóðveldi': 'TV'
    
}

In [355]:
for old, new in tqdm(party_map.items()):
    df['party'] = df['party'].str.replace(old, new, regex=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [356]:
df['party'].value_counts()

S                64475
V                57664
DF               40973
EL               35267
SF               28594
KF               23130
RV               21643
LA               19203
ALT               8758
NB                6525
V,UFG             2348
UFG               1875
V,UFG,M           1726
SF,S              1402
IA                1290
KD                 813
FG                 643
SIU                510
JF                 505
T                  287
SP                 286
SF,RV,S            190
LA,UFG             139
DD                 106
RV,UFG,ALT,FG      102
NQ                  97
TF                  65
M                   37
Name: party, dtype: int64

In [357]:
df.shape

(1030988, 37)

In [359]:
with open('data/clean/combined/df_combined.p', 'wb') as p:
    pickle.dump(df, p)

## Impute missing party affiliations

In [398]:
with open('data/clean/combined/df_combined.p', 'rb') as p:
    df = pickle.load(p)

In [304]:
# TODO: Impute parties based on last known value for each individual politician
# Set party for each party page based on mapping
# Set type for party pages on twitter; rest is "politician"
# Filter out any non-MPs based on wiki list
# Add labels for political leaders

In [433]:
party_handles = [
    'KonservativeDK',
    'KU_DK',
    'radikale',
    'radiaklungdom',
    'LibaralAlliance',
    'LiberalUngdomDK',
    'SFpolitik',
    'SF_Ungdom',
    'Enhedslisten',
    'sufnet',
    'venstredk',
    'VUngdom',
    'Spolitik',
    'DSU_1920',
    'DanskDf1995',
    'DFUngdom',
    'alternativet_',
    'AlternativeUnge',
    'NyeBorgerlige',
    'gronalliance',
    'KDpolitik',
    'moderaterne_dk',
    'friegronne'
]

In [431]:
df.loc[df['tw_handle'].isin(party_handles), ['level']] = 'party'

In [481]:
df_mps_parties = df.loc[
    df['full_name'].isin(mps['name']) |
    (df['level'] == 'party'),
].reset_index(drop = True)

In [482]:
df_mps_parties.shape

(579301, 37)

In [483]:
df_mps_parties.loc[
    (df_mps_parties['level'] == 'party') &
    df_mps_parties['party'].isna(),
    'party'
]

127237    NaN
127238    NaN
127239    NaN
127240    NaN
127241    NaN
         ... 
336329    NaN
336330    NaN
336331    NaN
336332    NaN
336333    NaN
Name: party, Length: 48411, dtype: object

In [443]:
df_mps_parties['party']

16629     NaN
16630     NaN
16631     NaN
16632     NaN
16633     NaN
         ... 
302583      S
302584     SF
302585      S
302586     SF
302587      S
Name: party, Length: 579301, dtype: object

In [523]:
def replace_name_with_party(name, party_map):
    for party, pattern in party_map.items():
        if name in pattern:
            name = name.replace(name, party)
    
    return name

In [525]:
replace_name_with_party('Siumut', party_map)

'SIU'

In [484]:
names_wo_parties = (
    df_mps_parties
        .loc[
                df_mps_parties['party'].isna() &
                (df_mps_parties['level'] == 'party'),
                'full_name'
        ]
    .progress_apply(lambda x: replace_name_with_party(x, party_map))
)

  0%|          | 0/48411 [00:00<?, ?it/s]

In [485]:
names_wo_parties

127237    SF
127238    SF
127239    SF
127240    SF
127241    SF
          ..
336329    SF
336330    SF
336331    SF
336332    SF
336333    SF
Name: full_name, Length: 48411, dtype: object

In [489]:
df_mps_parties[['full_name','party']]

,full_name,party
0,Dennis Flydtkjær,NaN
1,Dennis Flydtkjær,NaN
2,Dennis Flydtkjær,NaN
3,Dennis Flydtkjær,NaN
4,Dennis Flydtkjær,NaN
...,...,...
579296,Mette Frederiksen,S
579297,Anne Valentina Berthelsen,SF
579298,Mette Frederiksen,S
579299,Anne Valentina Berthelsen,SF


In [527]:
df_mps_parties['party'] = df_mps_parties['party'].fillna(names_wo_parties)

In [528]:
df_mps_parties['party'].value_counts(dropna = False)

NaN              276836
S                 57052
V                 52899
DF                41979
EL                31659
SF                23494
LA                21309
KF                18404
RV                17713
ALT               15725
NB                 6853
FG                 5514
V,UFG              2348
V,UFG,M            1726
UFG                1566
SF,S               1402
KD                  678
JF                  480
SIU                 378
SP                  280
M                   222
SF,RV,S             190
DD                  149
LA,UFG              139
RV,UFG,ALT,FG       102
NQ                   97
TF                   65
T                    26
K                    16
Name: party, dtype: int64

In [463]:
party_map = {
    'RV': ['Radikale Venstre'],
    'V': ['Venstre', 'Venstres Ungdom'],
    'SF': ['Socialistisk Folkeparti','SF Ungdom'],
    'KF': ['Det Konservative Folkeparti','Konservativ Ungdom'],
    'S': ['Socialdemokratiet','DSU'],
    'NB': ['Nye Borgerlige'],
    'LA': ['Liberal Alliance','Liberal Alliances Ungdom'],
    'DF': ['Dansk Folkeparti','Dansk Folkepartis Ungdom'],
    'EL': ['Enhedslisten','SUF - Socialistisk Ungdomsfront'],
    'ALT': ['Alternativet','Alternativets Unge','Grøn Alliance'],
    'FG': ['Frie Grønne'],
    'DD': ['Danmarksdemokraterne - Inger Støjberg'],
    'K': ['Kristendemokraterne'],
    'M': ['Moderaterne'],
    'IA': ['Inuit Ataqatigiit'],
    'SIU': ['Siumut'],
    'SP': ['Sambandsflokkurin'],
    'JF': ['Javnaðarflokkurin'],
    'TV': ['Tjóðveldi']
}

In [530]:
mps['group_name'].value_counts()

S      60
V      48
DF     38
EL     19
RV     16
SF     16
LA     14
KF     13
ALT    11
NB      4
SIU     2
IA      1
SP      1
TV      1
JF      1
Name: group_name, dtype: int64

In [531]:
name2party_default = mps.set_index('name').rename(columns = {'group_name': 'party'})[['party']]

In [532]:
df_party_imp = df_mps_parties.copy()

In [562]:
df_party_imp.loc[df_party_imp['level'].isna(), 'level'] = 'politician'

In [546]:
df_party_imp = (
    df_mps_parties
        .set_index('full_name')
        .combine_first(name2party_default)
        .reset_index()
        .rename(columns = {'index': 'full_name'})
)

In [496]:
# TODO: Check MPs that switched parties and correct party variable for relevant timebins

In [566]:
df_party_imp['start_time'] = pd.to_datetime(df_party_imp['start_time'], utc = True)

In [572]:
df_party_imp.loc[
    (df_party_imp['start_time'] > '2018-07-05 00:00:00'),
    [
    'doc', 'source', 'start_time',
    'full_name', 'level', 'party'
    ]
].drop_duplicates(subset='full_name')

,doc,source,start_time,full_name,level,party
4,[✨Sapiillutit inuiaqatigiinni peqataasarit.✨\n...,meta,2022-09-18 00:00:00+00:00,Aki-Matilda Høegh-Dam,politician,SIU
622,Mange tak. Den årlige forhandling om en finans...,parliament,2018-09-06 17:09:41+00:00,Aleqa Hammond,politician,NQ
1774,"Tak. Det åbne spørgsmål er jo, hvad Socialdemo...",parliament,2018-10-11 15:57:23+00:00,Alex Ahrendtsen,politician,DF
2240,Det offentlige forbrug er steget med 9 mia. kr...,twitter,2022-09-19 11:14:52+00:00,Alex Vanopslagh,politician,LA
6922,RT @TFKortholm: Vi er klar til valgkampen 💚🖖 S...,twitter,2022-09-19 16:55:24+00:00,Alternativet,party,ALT
...,...,...,...,...,...,...
565981,RT @AndreasBachM: 75.135 ‼️ gange tak til alle...,twitter,2022-09-19 13:45:33+00:00,Venstre,party,V
571351,En borgerlig statsminister forudsætter et borg...,twitter,2022-08-15 08:32:31+00:00,Venstres Ungdom,party,V
571498,[Jeg stiller op til bestyrelsesvalget i SK For...,meta,2022-03-25 00:00:00+00:00,Villum Christensen,politician,LA
572326,"@JackPGB Ja, man får det dårligt 😔",twitter,2022-09-18 19:52:19+00:00,Yildiz Akdogan,politician,S


In [581]:
df_party_imp.loc[
    (df_party_imp['start_time'] > '2018-07-05 00:00:00'),
    'source'
].value_counts()

twitter       227357
parliament     92515
meta           29722
Name: source, dtype: int64

In [576]:
df_party_imp.loc[
    (df_party_imp['start_time'] > '2018-07-05 00:00:00'),
    [
    'doc', 'source', 'start_time',
    'full_name', 'level', 'party'
    ]
].reset_index().to_pickle('data/clean/combined/subset_party_imputed.p')

____

## Fix MPs who switched parties *during* a given election period

In [5]:
df_temp = pd.read_pickle('data/clean/combined/subset_party_imputed.p')

In [6]:
df_temp['start_time'] = pd.to_datetime(df_temp['start_time']).dt.date

In [7]:
names_to_fix = df_temp.loc[
    df_temp['party'].str.contains(','),
    'full_name'
].drop_duplicates().values

In [8]:
print(names_to_fix)

['Astrid Krag' 'Inger Støjberg' 'Jesper Petersen' 'Lars Løkke Rasmussen'
 'Simon Emil Ammitzbøll-Bille']


In [9]:
def fix_single_affilitation(df, politician_name, party, period: 'tuple of dates, both inclusive'):
    df.loc[
        (df['full_name'] == politician_name) &
        (df['start_time'] >= pd.Timestamp(period[0]).date()) &
        (df['start_time'] <= pd.Timestamp(period[1]).date()),
        'party'
    ] = party
    
    return df

def fix_all_affiliations(df, politician_name, affiliation_dict):
    for party, period in affiliation_dict.items():   
        print(f'Ascribed party {party} to {politician_name} from {period[0]} to {period[1]}')
        
        fix_single_affilitation(
            df,
            politician_name,
            party,
            period
        )

In [10]:
AstridKrag_parties = {
    'SF': ('2007-10-03', '2014-02-03'),
    'S': ('2014-02-04', '2022-11-19')
}

IngerStøjberg_parties = {
    'V': ('2001-11-20', '2021-02-04'),
    'UFG': ('2021-02-05', '2021-12-21'),
    'DD': ('2022-11-01', '2022-11-19')
}

JesperPetersen_parties = {
    'SF': ('2007-11-13', '2013-03-20'),
    'S': ('2013-03-21', '2022-11-19')
}

LarsLøkke_parties = {
    'V': ('1994-09-21', '2020-12-31'),
    'UFG': ('2021-01-01', '2022-05-18'),
    'M': ('2022-05-19', '2022-11-19')
}

SimonEmilAmmitzbøll_parties = {
    'RV': ('2005-02-08', '2008-10-07'),
    'UFG': ('2008-10-08', '2009-06-15'),
    'LA': ('2009-06-16', '2019-10-22'),
    'UFG': ('2019-10-23', '2022-11-01')
}


BentBøgsted_parties = {
    'DF': ('2001-11-20', '2022-02-21'),
    'UFG': ('2022-02-22', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-01')
}

HKSkibby_parties = {
    'DF': ('2005-02-08', '2022-02-21'),
    'UFG': ('2022-02-22', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19')
}

KarinaAdsbøl_parties = {
    'DF': ('2011-09-15', '2022-02-21'),
    'UFG': ('2022-02-22', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19')
}

LiseBech_parties = {
    'DF': ('2015-06-18', '2022-02-21'),
    'UFG': ('2022-02-22', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19')
}

LiselotteBlixt_parties = {
    'DF': ('2007-11-13', '2022-02-21'),
    'UFG': ('2022-02-22', '2022-11-01')
}

MarieKrarup_parties = {
    'DF': ('2011-09-05', '2022-02-26'),
    'UFG': ('2022-02-27', '2022-11-01')
}

JHThulesenDahl_parties = {
    'DF': ('2011-09-15', '2022-06-24'),
    'UFG': ('2022-06-25', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19')
}

PeterSkaarup_parties = {
    'DF': ('1998-03-11', '2022-06-24'),
    'UFG': ('2022-06-25', '2022-07-28'),
    'DD': ('2022-07-29', '2022-11-19')
}

SørenEspersen_parties = {
    'DF': ('2005-02-08', '2022-06-24'),
    'UFG': ('2022-06-25', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19')   
}

DennisFlydtkjær_parties = {
    'DF': ('2011-09-15', '2022-06-25'),
    'UFG': ('2022-06-26', '2022-08-17'),
    'DD': ('2022-08-18', '2022-11-19') 
}

KristianThulesenDahl_parties = {
    'DF': ('1995-10-06', '2022-06-29'),
    'UFG': ('2022-06-30', '2022-07-31') 
}

MarcusKnuth_parties = {
    'V': ('2015-06-18', '2019-11-28'),
    'KF': ('2022-11-29', '2022-11-01') 
}

BrittBager_parties = {
    'V': ('2015-06-18', '2021-03-23'),
    'KF': ('2022-03-24', '2022-11-01') 
}

JensRohde_parties = {
    'V': ('1998-03-11', '2007-01-09'),
    'RV': ('2019-06-15', '2021-01-26'),
    'UFG': ('2021-02-27', '2021-04-26'),
    'KD': ('2021-04-27', '2022-08-25'),
    'UFG': ('2021-08-26', '2022-11-01')
}

IdaAuken_parties = {
    'SF': ('2007-11-13', '2014-02-03'),
    'RV': ('2014-02-04', '2021-02-02'),
    'S': ('2021-02-03', '2022-11-19')
}

UffeElbæk_parties = {
    'RV': ('2011-09-15', '2013-09-16'),
    'UFG': ('2013-09-17', '2015-03-12'),
    'ALT': ('2015-03-13', '2020-03-09'),
    'UFG': ('2020-03-10', '2021-11-02'),
    'FG': ('2021-11-03', '2022-09-16'),
    'ALT': ('2022-09-17', '2022-11-01')
}

SikandarSiddique_parties = {
    'ALT': ('2019-06-05', '2020-03-09'),
    'UFG': ('2020-03-10', '2021-11-02'),
    'FG': ('2021-11-03', '2022-11-01')
}

SusanneZimmer_parties = {
    'ALT': ('2019-06-05', '2020-03-09'),
    'UFG': ('2020-03-10', '2021-11-02'),
    'FG': ('2021-11-03', '2022-11-01')
}

RasmusNordqvist_parties = {
    'ALT': ('2015-06-18', '2020-03-09'),
    'UFG': ('2020-03-10', '2020-05-12'),
    'SF': ('2020-05-13', '2022-11-01')
}

OrlaØsterby_parties = {
    'KF': ('2016-12-05', '2020-12-03'),
    'UFG': ('2020-12-04', '2022-11-01')
}

NaserKhader_parties = {
    'LA': ('2007-07-10', '2009-01-04'),
    'UFG': ('2009-01-05', '2009-03-17'),
    'KF': ('2009-03-18', '2021-08-18'),
    'UFG': ('2021-08-19', '2022-11-01')
}

fixed_affiliations = {
    'Astrid Krag': AstridKrag_parties,
    'Inger Støjberg': IngerStøjberg_parties,
    'Jesper Petersen': JesperPetersen_parties,
    'Lars Løkke Rasmussen': LarsLøkke_parties,
    'Simon Emil Ammitzbøll-Bille': SimonEmilAmmitzbøll_parties,
    'Bent Bøgsted': BentBøgsted_parties,
    'Hans Kristian Skibby': HKSkibby_parties,
    'Karina Adsbøl': KarinaAdsbøl_parties,
    'Lise Bech': LiseBech_parties,
    'Liselotte Blixt': LiselotteBlixt_parties,
    'Jens Henrik Thulesen Dahl': JHThulesenDahl_parties,
    'Peter Skaarup': PeterSkaarup_parties,
    'Søren Espersen': SørenEspersen_parties,
    'Dennis Flydtkjær': DennisFlydtkjær_parties,
    'Kristian Thulesen Dahl': KristianThulesenDahl_parties,
    'Marcus Knuth': MarcusKnuth_parties,
    'Britt Bager': BrittBager_parties,
    'Jens Rohde': JensRohde_parties,
    'Ida Auken': IdaAuken_parties,
    'Uffe Elbæk': UffeElbæk_parties,
    'Sikandar Siddique': SikandarSiddique_parties,
    'Susanne Zimmer': SusanneZimmer_parties,
    'Rasmus Nordqvist': RasmusNordqvist_parties,
    'Orla Østerby': OrlaØsterby_parties,
    'Naser Khader': NaserKhader_parties
}

In [11]:
for name, affiliations in tqdm(fixed_affiliations.items()):
    fix_all_affiliations(df_temp, name, affiliations)

  0%|          | 0/25 [00:00<?, ?it/s]

Ascribed party SF to Astrid Krag from 2007-10-03 to 2014-02-03
Ascribed party S to Astrid Krag from 2014-02-04 to 2022-11-19
Ascribed party V to Inger Støjberg from 2001-11-20 to 2021-02-04
Ascribed party UFG to Inger Støjberg from 2021-02-05 to 2021-12-21
Ascribed party DD to Inger Støjberg from 2022-11-01 to 2022-11-19
Ascribed party SF to Jesper Petersen from 2007-11-13 to 2013-03-20
Ascribed party S to Jesper Petersen from 2013-03-21 to 2022-11-19
Ascribed party V to Lars Løkke Rasmussen from 1994-09-21 to 2020-12-31
Ascribed party UFG to Lars Løkke Rasmussen from 2021-01-01 to 2022-05-18
Ascribed party M to Lars Løkke Rasmussen from 2022-05-19 to 2022-11-19
Ascribed party RV to Simon Emil Ammitzbøll-Bille from 2005-02-08 to 2008-10-07
Ascribed party UFG to Simon Emil Ammitzbøll-Bille from 2019-10-23 to 2022-11-01
Ascribed party LA to Simon Emil Ammitzbøll-Bille from 2009-06-16 to 2019-10-22
Ascribed party DF to Bent Bøgsted from 2001-11-20 to 2022-02-21
Ascribed party UFG to Bent 

In [12]:
df_temp.loc[df_temp['party'] == 'K', 'party'] = 'KD'

In [15]:
df_temp['party'].value_counts(dropna = False)


S      80003
V      58671
EL     37903
SF     33652
RV     28769
DF     26535
KF     22295
LA     18853
ALT    14528
FG     10668
NB      8939
UFG     6743
KD       935
SIU      269
M        222
JF       216
DD       193
SP       157
NQ        43
Name: party, dtype: int64

In [16]:
# Unpack docs in list format
def unpack_listed_doc(doc):
    
    if type(doc) == list:
        out_doc = ''.join(doc)
    else:
        out_doc = doc
    
    return out_doc

In [17]:
df_temp['doc'] = df_temp['doc'].progress_apply(lambda x: unpack_listed_doc(x))

  0%|          | 0/349594 [00:00<?, ?it/s]

In [18]:
df_temp.to_pickle('data/clean/combined/subset_party_imputed_v2.p')

_____

## `FuzzyJoin` test (OLD)

In [7]:
with open('data/clean/twitter/full_tweet_df_19-09-22.p', 'rb') as p:
    tweet_df = pickle.load(p)

In [13]:
ad_files = ['politician_ads_v1']#, 'party_ads_v1']
ad_dfs = []

for ad_file in ad_files:
    with open(f'data/clean/meta/{ad_file}.p', 'rb') as p:
        ad_dfs.append(pickle.load(p))

ad_df = pd.concat(ad_dfs).reset_index(drop = True)

In [15]:
print(ad_df['page_name'].head())
print(tweet_df['screen_name'].head())

0    Andreas Steenberg
1    Andreas Steenberg
2    Andreas Steenberg
3    Andreas Steenberg
4    Andreas Steenberg
Name: page_name, dtype: object
0    AndersKuhnau
1    AndersKuhnau
2    AndersKuhnau
3    AndersKuhnau
4    AndersKuhnau
Name: screen_name, dtype: object


In [2]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=75, limit=2):

    s = df_2[key2].tolist()
    
    m = df_1[key1].progress_apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].progress_apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [58]:
twitter_names = tweet_df[['screen_name']].drop_duplicates()
meta_names = ad_df[['page_name']].drop_duplicates()

In [59]:
fuzzy_test = fuzzy_merge(twitter_names, meta_names, 'screen_name', 'page_name')

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

In [61]:
fuzzy_test.loc[fuzzy_test['matches'] != ''].tail(50)

,screen_name,matches
4032,simonkollerup,Simon Kollerup
7369,stinuslindgreen,Stinus Lindgreen - Radikale Venstre
10569,KirstenNormann,Kirsten Normann Andersen
27849,MadsAndersenC,Hans Andersen
28017,SorenPape,Søren Pape Poulsen
30838,SorenEggeRasmus,Søren Egge Rasmussen
33595,RasmusPrehn,Rasmus Prehn
36796,MarieKrarup,Marie Krarup
39173,AlexVanopslagh,Alex Vanopslagh
51719,Kristianthdahl,Kristian Thulesen Dahl


In [5]:
fuzz.ratio('Søren Pape Poulsen', 'Søren Pape')

71

In [ ]:
p